In [1]:
import os
import json
import pandas as pd
import PyPDF2
import json
import constants

from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
from dotenv import load_dotenv

In [2]:
load_dotenv()
apikey = os.environ["OPENAI_KEY"]

In [3]:
# Initialize the ChatOpenAI object with the specified parameters
llm = ChatOpenAI(openai_api_key=apikey, model_name="gpt-3.5-turbo", temperature=0.7)

d:\Pyhton\genie-mcq\venv\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [4]:
PATH = "D:\Pyhton\genie-mcq\Response.json"
with open(PATH,"r") as f:
    RESPONSE_JSON=json.load(f)

In [5]:
mcq_prompt = PromptTemplate(
    input_variables=["text","number","subject","tone","RESPONSE_JSON"],
    template=constants.TEMPLATE,
)

mcq_chain = LLMChain(llm=llm,prompt=mcq_prompt,output_key="mcq",verbose=True)

In [6]:
review_prompt = PromptTemplate(
        input_variables=["subject","mcq"],
        template=constants.TEMPLATE2
)

review_chain = LLMChain(llm=llm,prompt=review_prompt,output_key="review",verbose=True)

In [7]:
generate_evaluate_chain=SequentialChain(chains=[mcq_chain, review_chain], input_variables=["text", "number", "subject", "tone", "RESPONSE_JSON"],output_variables=["mcq", "review"], verbose=True,)

In [8]:
PATH = "D:\Pyhton\genie-mcq\data.txt"
with open(PATH,"r") as file:
    TEXT=file.read()

In [9]:
TEXT
NUMBER = 2
SUBJECT = "History"
TONE = "easy"

In [10]:
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
        "text": TEXT,
        "number": NUMBER,
        "subject": SUBJECT,
        "tone": TONE,
        "RESPONSE_JSON": json.dumps(RESPONSE_JSON)
        }
    )


d:\Pyhton\genie-mcq\venv\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:The Society was founded by Dr. S.P. Gupta (a prominent archaeologist and Chairman of the Indian Archaeological Society) and Dr. D. Devahuti of the National Museum, New Delhi, and inaugurated by then Prime Minister Morarji Desai accompanied by eminent scholars such as Prof. Lallanji Gopal (of BHU), Prof. B. P. Sinha (head of History at Patna University), Prof. Khalid Ahmed Nizami (head of History at Aligarh Muslim University), Prof. G.C. Pande, Prof. K.D. Bajpai, and Prof. Tan Chung.

The publication History Today was first edited by Dr. S.K Jolly (a historian of Sikh Revivalism) and subsequently, in due course it was edited by Dr. Vandana Kaushik, Ms. S. Radhakrishnan and Prof. D.P. Tiwari.

When the Janata government of 1977-79 (including the former Jan Sangh) was elected, it blocked the Towards Freedom Project of the Indian Council of Historical Research (ICHR) claiming that th

In [11]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1169
Total Cost:0.001874


In [12]:
quiz = response.get("mcq").strip()
quiz = quiz.replace('### RESPONSE_JSON\n', '')
quiz

'{"questions": {"1": {"no": "1", "ques": "Who founded The Society mentioned in the text?", "options": {"a": "Dr. S.P. Gupta and Dr. D. Devahuti", "b": "Prof. Lallanji Gopal and Prof. B. P. Sinha", "c": "Prof. Khalid Ahmed Nizami and Prof. G.C. Pande", "d": "Prof. K.D. Bajpai and Prof. Tan Chung"}}, "2": {"no": "2", "ques": "Which government blocked the Towards Freedom Project of the ICHR?", "options": {"a": "Congress government", "b": "Janata government", "c": "BJP government", "d": "Communist government"}}}, "answers": {"1": {"ans": "a"}, "2": {"ans": "b"}}}'

In [13]:
try:
    quiz_json = json.loads(quiz)
    questions = quiz_json["questions"]
    answers = quiz_json["answers"]

    # Process the JSON data further
    print("\nQuestions:")
    for q_no, details in questions.items():
        print(f"Q{q_no}: {details['ques']}")
        for option, text in details["options"].items():
                print(f"{option}: {text}")
    print("\nAnswers:")
    for q_no, ans in answers.items():
        print(f"Answer {q_no}: {ans}")
except json.JSONDecodeError as e:
    print("Error decoding JSON:", e)




Questions:
Q1: Who founded The Society mentioned in the text?
a: Dr. S.P. Gupta and Dr. D. Devahuti
b: Prof. Lallanji Gopal and Prof. B. P. Sinha
c: Prof. Khalid Ahmed Nizami and Prof. G.C. Pande
d: Prof. K.D. Bajpai and Prof. Tan Chung
Q2: Which government blocked the Towards Freedom Project of the ICHR?
a: Congress government
b: Janata government
c: BJP government
d: Communist government

Answers:
Answer 1: {'ans': 'a'}
Answer 2: {'ans': 'b'}
